In [ ]:
%sql
WITH summary AS (
    SELECT
        providertaxid,
        providernpi,
        sel_category,
        paid_amount_bucket,
        SUM(CASE WHEN status = 'Findings' THEN 1 ELSE 0 END) AS findings,
        SUM(CASE WHEN status = 'NoFindings' THEN 1 ELSE 0 END) AS nofindings
    FROM prod_aupsreportinghuboffshore_gold.aetnaportingmart_audre
    WHERE auditdate BETWEEN '2025-01-01' AND '2025-11-30'
      AND providertaxid IS NOT NULL
      AND TRIM(providertaxid) <> ''
    GROUP BY providertaxid, providernpi, sel_category, paid_amount_bucket
)

SELECT
    providertaxid,
    providernpi,
    sel_category,
    paid_amount_bucket,
    findings + nofindings AS audits,
    ROUND(findings * 1.0 / NULLIF((findings + nofindings),0), 4) AS hitrate,

    CASE
        WHEN (findings + nofindings) > 100
         AND (findings * 1.0 / NULLIF((findings + nofindings),0)) < 0.08
             THEN 'Very low hitrate — High audits with low value. Remove / avoid selecting here'
        WHEN (findings + nofindings) > 100
         AND (findings * 1.0 / NULLIF((findings + nofindings),0)) BETWEEN 0.08 AND 0.12
             THEN 'Medium hitrate — Consider selective audits. Monitor performance'
        WHEN (findings + nofindings) > 100
         AND (findings * 1.0 / NULLIF((findings + nofindings),0)) > 0.12
             THEN 'Good hitrate — Strong performer. Prioritize this category'
        ELSE 'Low audit volume — Not enough data to evaluate'
    END AS interpretation

FROM summary
WHERE (findings + nofindings) > 100
ORDER BY hitrate ASC;


In [ ]:
WITH summary AS (
    SELECT
        providertaxid,
        providernpi,
        sel_category,
        SUM(CASE WHEN status = 'Findings' THEN 1 ELSE 0 END) AS findings,
        SUM(CASE WHEN status = 'NoFindings' THEN 1 ELSE 0 END) AS nofindings
    FROM prod_aupsreportinghuboffshore_gold.aetnaportingmart_audre
    WHERE auditdate BETWEEN '2025-01-01' AND '2025-11-30'
    GROUP BY providertaxid, providernpi, sel_category
)

SELECT
    providertaxid,
    providernpi,
    sel_category,
    findings + nofindings AS audits,
    ROUND(findings * 1.0 / NULLIF((findings + nofindings),0), 4) AS hitrate,

    CASE
        WHEN (findings + nofindings) > 100
         AND (findings * 1.0 / NULLIF((findings + nofindings),0)) > 0.12
            THEN 'Strong performance — Good audits & high hitrate. Keep selecting in this category'
        WHEN (findings + nofindings) > 100
         AND (findings * 1.0 / NULLIF((findings + nofindings),0)) BETWEEN 0.08 AND 0.12
            THEN 'Moderate performance — Monitor & use selectively'
        WHEN (findings + nofindings) > 100
         AND (findings * 1.0 / NULLIF((findings + nofindings),0)) < 0.08
            THEN 'Poor performance — High audits but low hitrate. Remove from this category'
        ELSE 'Low audits — Insufficient volume to evaluate'
    END AS interpretation

FROM summary
WHERE (findings + nofindings) > 100
ORDER BY providertaxid, providernpi, hitrate DESC;


In [ ]:
WITH summary AS (
    SELECT
        providertaxid,
        providernpi,
        sel_category,
        SUM(CASE WHEN status = 'Findings' THEN 1 ELSE 0 END) AS findings,
        SUM(CASE WHEN status = 'NoFindings' THEN 1 ELSE 0 END) AS nofindings
    FROM prod_aupsreportinghuboffshore_gold.aetnaportingmart_audre
    WHERE auditdate BETWEEN '2025-01-01' AND '2025-11-30'
    GROUP BY providertaxid, providernpi, sel_category
)

SELECT
    providertaxid,
    providernpi,
    sel_category,
    findings + nofindings AS audits,
    ROUND(findings * 1.0 / NULLIF((findings + nofindings),0), 4) AS hitrate,

    CASE
        WHEN (findings + nofindings) > 200
         AND (findings * 1.0 / NULLIF((findings + nofindings),0)) > 0.12
            THEN 'Strong Performer — High audits & high hitrate. Prioritize selection'
        WHEN (findings + nofindings) BETWEEN 50 AND 200
         AND (findings * 1.0 / NULLIF((findings + nofindings),0)) > 0.12
            THEN 'Emerging Opportunity — Low/medium audits but high hitrate. Increase selection'
        WHEN (findings + nofindings) > 200
         AND (findings * 1.0 / NULLIF((findings + nofindings),0)) BETWEEN 0.08 AND 0.12
            THEN 'Stable Performer — Balanced return. Maintain audit volume'
        WHEN (findings + nofindings) > 200
         AND (findings * 1.0 / NULLIF((findings + nofindings),0)) < 0.08
            THEN 'Over Audited / Low Value — High audits but poor hitrate. Reduce / remove from category'
        WHEN (findings + nofindings) BETWEEN 50 AND 200
         AND (findings * 1.0 / NULLIF((findings + nofindings),0)) BETWEEN 0.08 AND 0.12
            THEN 'Monitor Zone — Medium audits medium hitrate. Watch performance'
        WHEN (findings + nofindings) < 50
            THEN 'Needs More Data — Low audits cannot conclude'
        ELSE 'Not Classified'
    END AS interpretation

FROM summary
ORDER BY providertaxid, providernpi, hitrate DESC;


In [ ]:
WITH summary AS (
    SELECT
        providertaxid,
        providernpi,
        sel_category,
        SUM(CASE WHEN status = 'Findings' THEN 1 ELSE 0 END) AS findings,
        SUM(CASE WHEN status = 'NoFindings' THEN 1 ELSE 0 END) AS nofindings
    FROM prod_aupsreportinghuboffshore_gold.aetnaportingmart_audre
    WHERE auditdate BETWEEN '2025-01-01' AND '2025-11-30'
    GROUP BY providertaxid, providernpi, sel_category
),

scored AS (
    SELECT
        providertaxid,
        providernpi,
        sel_category,
        (findings + nofindings) AS audits,
        ROUND(findings * 1.0 / NULLIF((findings + nofindings),0), 4) AS hitrate,

        RANK() OVER (
            PARTITION BY providertaxid, providernpi
            ORDER BY ROUND(findings * 1.0 / NULLIF((findings + nofindings),0), 4) DESC
        ) AS hitrate_rank,

        RANK() OVER (
            PARTITION BY providertaxid, providernpi
            ORDER BY (findings + nofindings) DESC
        ) AS audit_rank,

        AVG(ROUND(findings * 1.0 / NULLIF((findings + nofindings),0), 4))
        OVER (PARTITION BY providertaxid, providernpi) AS avg_hitrate
    FROM summary
)

SELECT
    providertaxid,
    providernpi,
    sel_category,
    audits,
    hitrate,

    CASE
        WHEN hitrate_rank = 1 AND audit_rank = 1
            THEN 'Best Category — Highest hitrate & highest audit volume. Strongly recommended'
        WHEN hitrate_rank = 1
            THEN 'High performing category — Highest hitrate relative to others. Increase selection'
        WHEN hitrate_rank = 2
            THEN 'Moderate performer — Second best option for this provider'
        WHEN hitrate_rank = (SELECT MAX(hitrate_rank) FROM scored s2
                             WHERE s2.providertaxid = scored.providertaxid
                               AND s2.providernpi = scored.providernpi)
            THEN 'Weak category — Lowest hitrate. Consider removing'
        ELSE 'Needs evaluation — Middle range performer'
    END AS interpretation

FROM scored
ORDER BY providertaxid, providernpi, hitrate DESC;


In [ ]:
/* ===========================================================
   1. HITRATE BREAKDOWN ACROSS SEL_CATEGORY
   Dynamic interpretation vs other categories for same provider
   =========================================================== */

WITH summary AS (
    SELECT
        providertaxid,
        providernpi,
        sel_category,
        SUM(CASE WHEN status = 'Findings' THEN 1 ELSE 0 END) AS findings,
        SUM(CASE WHEN status = 'NoFindings' THEN 1 ELSE 0 END) AS nofindings
    FROM prod_aupsreportinghuboffshore_gold.aetnaportingmart_audre
    GROUP BY providertaxid, providernpi, sel_category
),
scored AS (
    SELECT
        providertaxid,
        providernpi,
        sel_category,
        (findings + nofindings) AS audits,
        ROUND(findings * 1.0 / NULLIF((findings + nofindings),0), 4) AS hitrate,

        RANK() OVER (PARTITION BY providertaxid, providernpi ORDER BY ROUND(findings * 1.0 / NULLIF((findings + nofindings),0), 4) DESC) AS hitrate_rank,
        RANK() OVER (PARTITION BY providertaxid, providernpi ORDER BY (findings + nofindings) DESC) AS audit_rank,

        AVG(ROUND(findings * 1.0 / NULLIF((findings + nofindings),0), 4)) OVER (PARTITION BY providertaxid, providernpi) AS avg_hitrate
    FROM summary
)

SELECT
    providertaxid,
    providernpi,
    sel_category,
    audits,
    hitrate,
    avg_hitrate,

    CASE
        WHEN hitrate_rank = 1 AND audit_rank = 1 THEN 'Best Fit Category — Max ROI. Highest hitrate & audit volume. Strongly recommended'
        WHEN hitrate_rank = 1 AND audit_rank > 1 THEN 'Emerging Strong Potential — Highest hitrate but low audits. Increase selection to unlock value'
        WHEN hitrate_rank = 2 THEN 'Moderate Performer — Second-best productivity zone'
        WHEN hitrate_rank = (SELECT MAX(hitrate_rank) FROM scored s2 WHERE s2.providertaxid=scored.providertaxid AND s2.providernpi=scored.providernpi) THEN 'Weak Category — Lowest hitrate. Reduce focus or remove'
        WHEN audits = 0 THEN 'No Insights — No audits for comparison'
        ELSE 'Stable Performer — Consistent but not leading'
    END AS interpretation

FROM scored
ORDER BY providertaxid, providernpi, hitrate DESC;


In [ ]:
/* ===========================================================
   2. HITRATE BREAKDOWN BY PAID AMOUNT BUCKET
   Dynamic relative ranking per provider
   =========================================================== */

WITH summary AS (
    SELECT
        providertaxid,
        providernpi,
        paid_amount_bucket,
        SUM(CASE WHEN status = 'Findings' THEN 1 ELSE 0 END) AS findings,
        SUM(CASE WHEN status = 'NoFindings' THEN 1 ELSE 0 END) AS nofindings
    FROM prod_aupsreportinghuboffshore_gold.aetnaportingmart_audre
    GROUP BY providertaxid, providernpi, paid_amount_bucket
),
scored AS (
    SELECT
        providertaxid,
        providernpi,
        paid_amount_bucket,
        (findings + nofindings) AS audits,
        ROUND(findings * 1.0 / NULLIF((findings + nofindings),0), 4) AS hitrate,

        RANK() OVER (PARTITION BY providertaxid, providernpi ORDER BY ROUND(findings * 1.0 / NULLIF((findings + nofindings),0), 4) DESC) AS hitrate_rank,
        RANK() OVER (PARTITION BY providertaxid, providernpi ORDER BY (findings + nofindings) DESC) AS audit_rank
    FROM summary
)

SELECT
    providertaxid,
    providernpi,
    paid_amount_bucket,
    audits,
    hitrate,

    CASE
        WHEN hitrate_rank = 1 AND audit_rank = 1 THEN 'High Value Bucket — Top financial return area. Maximize volume here'
        WHEN hitrate_rank = 1 AND audit_rank > 1 THEN 'Potential Goldmine — High hitrate. Expand effort'
        WHEN hitrate_rank = (SELECT MAX(hitrate_rank) FROM scored s2 WHERE s2.providertaxid=scored.providertaxid AND s2.providernpi=scored.providernpi) THEN 'Low ROI Bucket — Heavy auditing may not be effective'
        ELSE 'Neutral Performing Bucket — Balanced performance'
    END AS interpretation

FROM scored
ORDER BY providertaxid, providernpi, hitrate DESC;


In [ ]:
/* ===========================================================================
   3. COMBINATION LEVEL PERFORMANCE — BEST FINAL DECISION ENGINE
   To identify ideal category + bucket combination for selection strategy
   =========================================================================== */

WITH summary AS (
    SELECT
        providertaxid,
        providernpi,
        sel_category,
        paid_amount_bucket,
        SUM(CASE WHEN status = 'Findings' THEN 1 ELSE 0 END) AS findings,
        SUM(CASE WHEN status = 'NoFindings' THEN 1 ELSE 0 END) AS nofindings
    FROM prod_aupsreportinghuboffshore_gold.aetnaportingmart_audre
    GROUP BY providertaxid, providernpi, sel_category, paid_amount_bucket
),
scored AS (
    SELECT
        providertaxid,
        providernpi,
        sel_category,
        paid_amount_bucket,
        (findings + nofindings) AS audits,
        ROUND(findings * 1.0 / NULLIF((findings + nofindings),0), 4) AS hitrate,

        RANK() OVER (
            PARTITION BY providertaxid, providernpi
            ORDER BY ROUND(findings * 1.0 / NULLIF((findings + nofindings),0), 4) DESC
        ) AS hitrate_rank,

        RANK() OVER (
            PARTITION BY providertaxid, providernpi
            ORDER BY (findings + nofindings) DESC
        ) AS audit_rank
    FROM summary
)

SELECT
    providertaxid,
    providernpi,
    sel_category,
    paid_amount_bucket,
    audits,
    hitrate,

    CASE
        WHEN hitrate_rank = 1 AND audit_rank = 1 THEN 'Optimal Combination — Best success + highest volume. MUST prioritize'
        WHEN hitrate_rank = 1 THEN 'High ROI Combination — Increase selection volume immediately'
        WHEN audit_rank = 1 AND hitrate_rank > 2 THEN 'Over-Audited Combo — Heavy work, poor value. Reduce'
        WHEN hitrate_rank = (SELECT MAX(hitrate_rank) FROM scored s2 WHERE s2.providertaxid=scored.providertaxid AND s2.providernpi=scored.providernpi) THEN 'Worst Case — Candidate for removal'
        ELSE 'Monitor Category — Mid performer'
    END AS interpretation

FROM scored
ORDER BY providertaxid, providernpi, hitrate DESC;


In [ ]:
WITH summary AS (
    SELECT
        providertaxid,
        providernpi,
        sel_category,
        SUM(CASE WHEN status = 'Findings' THEN 1 ELSE 0 END) AS findings,
        SUM(CASE WHEN status = 'Nofindings' THEN 1 ELSE 0 END) AS nofindings,
        SUM(CASE WHEN recordreqdate IS NOT NULL THEN 1 ELSE 0 END) AS recordreq_count,
        SUM(CASE WHEN status = 'Findings' THEN findlettersent_overpay ELSE 0 END) AS op_amount
    FROM prod.appsreportinghuboffshore_gold.aetnareportingmart_azure
    WHERE auditdate BETWEEN '2025-01-01' AND '2025-11-30'
    GROUP BY providertaxid, providernpi, sel_category
),

scored AS (
    SELECT
        providertaxid,
        providernpi,
        sel_category,
        findings,
        nofindings,
        recordreq_count,
        op_amount,
        (findings + nofindings) AS audits,
        ROUND(findings * 1.0 / NULLIF((findings + nofindings),0), 4) AS hitrate,

        RANK() OVER (
            PARTITION BY providertaxid, providernpi
            ORDER BY ROUND(findings * 1.0 / NULLIF((findings + nofindings),0), 4) DESC
        ) AS hitrate_rank,

        RANK() OVER (
            PARTITION BY providertaxid, providernpi
            ORDER BY (findings + nofindings) DESC
        ) AS audit_rank,

        AVG(ROUND(findings * 1.0 / NULLIF((findings + nofindings),0), 4))
        OVER (PARTITION BY providertaxid, providernpi) AS avg_hitrate
    FROM summary
)

SELECT
    providertaxid,
    providernpi,
    sel_category,
    audits,
    findings,
    nofindings,
    recordreq_count,
    op_amount,
    hitrate,

    CASE
        WHEN audits < 15 THEN 'Insufficient audit volume — sample too small to evaluate performance'
        WHEN hitrate_rank = 1 AND audit_rank = 1
            THEN 'Best performing category — Highest hitrate & highest audits. Strong recommendation'
        WHEN hitrate_rank = 1
            THEN 'High performing category — Strong hitrate relative to peers. Increase focus'
        WHEN hitrate_rank = 2
            THEN 'Moderate performer — Consider secondary prioritization'
        WHEN hitrate_rank = (SELECT MAX(hitrate_rank) FROM scored s2
                             WHERE s2.providertaxid = scored.providertaxid
                               AND s2.providernpi = scored.providernpi)
            AND audits >= 15
            THEN 'Weak category — Lowest hitrate. Consider deprioritizing'
        ELSE 'Average performance — Further evaluation needed'
    END AS interpretation

FROM scored
ORDER BY providertaxid, providernpi, hitrate DESC;


In [ ]:
WITH summary AS (
    SELECT
        providertaxid,
        providernpi,
        paid_amount_bucket,
        SUM(CASE WHEN status = 'Findings' THEN 1 ELSE 0 END) AS findings,
        SUM(CASE WHEN status = 'Nofindings' THEN 1 ELSE 0 END) AS nofindings,
        SUM(CASE WHEN recordreqdate IS NOT NULL THEN 1 ELSE 0 END) AS recordreq_count,
        SUM(CASE WHEN status = 'Findings' THEN findlettersent_overpay ELSE 0 END) AS op_amount
    FROM prod.appsreportinghuboffshore_gold.aetnareportingmart_azure
    WHERE auditdate BETWEEN '2025-01-01' AND '2025-11-30'
    GROUP BY providertaxid, providernpi, paid_amount_bucket
),

scored AS (
    SELECT
        providertaxid,
        providernpi,
        paid_amount_bucket,
        findings,
        nofindings,
        recordreq_count,
        op_amount,
        (findings + nofindings) AS audits,
        ROUND(findings * 1.0 / NULLIF((findings + nofindings),0), 4) AS hitrate,

        RANK() OVER (
            PARTITION BY providertaxid, providernpi
            ORDER BY ROUND(findings * 1.0 / NULLIF((findings + nofindings),0), 4) DESC
        ) AS hitrate_rank,

        RANK() OVER (
            PARTITION BY providertaxid, providernpi
            ORDER BY (findings + nofindings) DESC
        ) AS audit_rank,

        AVG(ROUND(findings * 1.0 / NULLIF((findings + nofindings),0), 4))
        OVER (PARTITION BY providertaxid, providernpi) AS avg_hitrate
    FROM summary
)

SELECT
    providertaxid,
    providernpi,
    paid_amount_bucket,
    audits,
    findings,
    nofindings,
    recordreq_count,
    op_amount,
    hitrate,

    CASE
        WHEN audits < 15 THEN 'Insufficient audit volume — Cannot evaluate performance for this bucket'
        WHEN hitrate_rank = 1 AND audit_rank = 1
            THEN 'Best bucket — Highest hitrate & highest audits. Strong recommendation for this range'
        WHEN hitrate_rank = 1
            THEN 'High potential bucket — Strong hitrate. Increase focus on this range'
        WHEN hitrate_rank = 2
            THEN 'Moderate performer — Secondary bucket'
        WHEN hitrate_rank = (SELECT MAX(hitrate_rank) FROM scored s2
                             WHERE s2.providertaxid = scored.providertaxid
                               AND s2.providernpi = scored.providernpi)
            AND audits >= 15
            THEN 'Weak bucket — lowest hitrate. Consider deprioritization'
        ELSE 'Average bucket — needs further analysis'
    END AS interpretation

FROM scored
ORDER BY providertaxid, providernpi, hitrate DESC;
